In [1]:
import boto3
import json
import concurrent.futures

In [2]:
source_coop_credentials = {
    'region_name': 'us-west-2',
    'AWS_ACCESS_KEY_ID': 'xxx',
    'AWS_SECRET_ACCESS_KEY': 'xxx',
    'AWS_SESSION_TOKEN': 'xxx'
}

First get the file size using the credentials:

In [3]:
# Create a session from the dictionary
read_session = boto3.Session(
    aws_access_key_id=source_coop_credentials['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=source_coop_credentials['AWS_SECRET_ACCESS_KEY'],
    aws_session_token=source_coop_credentials['AWS_SESSION_TOKEN'],
    region_name=source_coop_credentials['region_name']
)


# Create an S3 client from the session
read_s3_client = read_session.client('s3')

# Use the S3 client to get object metadata
source_name = 'us-west-2.opendata.source.coop'
source_key = 'protomaps/openstreetmap/tiles/v3.pmtiles'
metadata = read_s3_client.head_object(Bucket=source_name, Key=source_key)
object_size = metadata['ContentLength']

print(f'Object Size: {object_size}')

Object Size: 127293340693


Now initialise a multipart upload

In [10]:
def invoke_lambda(lambda_function_name, read_credentials, source_name, source_key, upload_id, part_num, byte_position, part_size):
    lambda_client = boto3.client('lambda')
    response = lambda_client.invoke(
        FunctionName=lambda_function_name,
        InvocationType='RequestResponse',
        Payload=json.dumps({
            'credentials': read_credentials,
            'source_bucket': source_name,
            'source_key': source_key,
            'upload_id': upload_id,
            'part_num': part_num,
            'byte_position': byte_position,
            'part_size': part_size,
        })
    )
    
    response_payload = json.loads(response['Payload'].read())
    #return response_payload
    if response_payload['ResponseMetadata']['HTTPStatusCode'] == 200:
        return {'PartNumber': part_num, 'ETag': response_payload['ETag']}
    else:
        return {'PartNumber': part_num, 'ETag': 'Failed'}


initiate multipart upload

In [11]:
s3_client = boto3.client('s3')
init_response = s3_client.create_multipart_upload(Bucket='my-pmtiles-basemap', Key='v3.pmtiles')

In [12]:
upload_id = init_response['UploadId']

In [13]:
part_size = 1000 * 1024 * 1024  # Set your part size here, e.g., 1000 MB parts
lambda_function_name = 'MyPmtilesStack-FetchLatestPMtilesEFB78E2C-fq8HioDfVDCa'

In [14]:
num_parts = (object_size + part_size - 1) // part_size
num_parts

122

In [15]:
def invoke_lambda_wrapper(part_num):
    byte_position = part_num * part_size
    actual_part_size = min(part_size, object_size - byte_position)
    return invoke_lambda(
        lambda_function_name,
        source_coop_credentials,
        source_name,
        source_key,
        upload_id,
        part_num + 1,   # 1 indexed not 0
        byte_position,
        actual_part_size
    )

parts = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(invoke_lambda_wrapper, part_num) for part_num in range(num_parts)]
    for future in concurrent.futures.as_completed(futures):
        parts.append(future.result())

In [16]:
failed = []
for part in parts:
    if part['ETag'] == 'failed':
        failed.append(part)

if len(failed) > 0:
    print(failed)
    #break

In [21]:
sorted_parts = sorted(parts, key=lambda x: x['PartNumber'])

In [23]:
sorted_parts

[{'PartNumber': 1, 'ETag': '"6ec63934173788dddf5b4e06c1ea76cb"'},
 {'PartNumber': 2, 'ETag': '"8be2dd1f1abc5b3b2728abb7a1f3c8bf"'},
 {'PartNumber': 3, 'ETag': '"0d06639acac67805cbee376ab1b2b659"'},
 {'PartNumber': 4, 'ETag': '"67fb4700229e0770af6e32f16d3a4999"'},
 {'PartNumber': 5, 'ETag': '"4d8a5fd33eb4ad544ceebc40223d7d65"'},
 {'PartNumber': 6, 'ETag': '"573ea6cce2c44f047a888bde2f775672"'},
 {'PartNumber': 7, 'ETag': '"63dd999049c812b4d99886ae9ca67d4f"'},
 {'PartNumber': 8, 'ETag': '"65c338799f273fc7722b4cd22d077bf1"'},
 {'PartNumber': 9, 'ETag': '"85ec0ce1ef4a063fa6fae21405b4f68f"'},
 {'PartNumber': 10, 'ETag': '"a7078aa05649f735ff34ad22006f7932"'},
 {'PartNumber': 11, 'ETag': '"91ba64f94222b36a9a69cc2c65b3ff76"'},
 {'PartNumber': 12, 'ETag': '"3ec8e1ee7a52073c6995a72df904aa76"'},
 {'PartNumber': 13, 'ETag': '"631fbc30226bf7347826e5c3bb622493"'},
 {'PartNumber': 14, 'ETag': '"a851665ed7b06e80947f67ad20b859bb"'},
 {'PartNumber': 15, 'ETag': '"957cf1d3bd2cfb04ffd8673d1e9ff913"'},
 {'P

Create parts to upload

In [24]:
response = s3_client.complete_multipart_upload(
    Bucket='my-pmtiles-basemap',
    Key='v3.pmtiles',
    UploadId=upload_id,
    MultipartUpload={
        'Parts': sorted_parts
    }
)

In [25]:
response

{'ResponseMetadata': {'RequestId': 'R1JK56H5T91BQHHA',
  'HostId': '0qhr/Q35Zr1Fgjq5iHBYOvloGScUjoTL+wWOx8Jao/bWXgCSfkrCwNYZZ/hog0HqDXbNdRYTTIw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '0qhr/Q35Zr1Fgjq5iHBYOvloGScUjoTL+wWOx8Jao/bWXgCSfkrCwNYZZ/hog0HqDXbNdRYTTIw=',
   'x-amz-request-id': 'R1JK56H5T91BQHHA',
   'date': 'Sun, 04 Aug 2024 17:26:09 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ServerSideEncryption': 'AES256',
 'Location': 'https://my-pmtiles-basemap.s3.us-west-2.amazonaws.com/v3.pmtiles',
 'Bucket': 'my-pmtiles-basemap',
 'Key': 'v3.pmtiles',
 'ETag': '"d1d05ab354d14aa36670f4fa6d1e3d91-122"'}

In [24]:
#response = s3_client.abort_multipart_upload(Bucket='my-pmtiles-basemap', Key='v3.pmtiles' ,UploadId=upload_id)

In [26]:
#response